In [ ]:
# https://www.bfs.admin.ch/bfs/fr/home/services/recherche/api.html
# https://dam-api.bfs.admin.ch/hub/swagger-ui/index.html
# communes
# https://www.bfs.admin.ch/bfs/fr/home/services/recherche/api/api-gemeinde.assetdetail.15224055.html

# motorisation
# https://www.pxweb.bfs.admin.ch/sq/ccc96ddd-61a9-49ab-ad7f-dabbc934e8c0

import requests

# Population
dataset_url = "https://www.pxweb.bfs.admin.ch/api/v1/en/px-x-0102030000_101/px-x-0102030000_101.px"

metadata_response = requests.get(dataset_url)
metadata_json = metadata_response.json()

In [ ]:
print(metadata_json["title"])
for v in metadata_json["variables"]:
    print("---")
    print(f"{v['text']} ({v['code']})")
    print(f"{v['valueTexts']} ({v['values']})")

In [ ]:
response2 = requests.post(
    dataset_url,
    json={
        "query": [
            {
                "code": "Alter",
                "selection": {
                    "filter": "item",
                    "values": metadata_json["variables"][1]["values"],
                },
            }
        ],
        "response": {"format": "json-stat"},
    },
)
json = response2.json()

In [ ]:
import pandas as pd

values = {"values": json["dataset"]["value"]}
length = 1
total_length = len(json["dataset"]["value"])
for dimension_id in json["dataset"]["dimension"]["id"]:
    dimension = json["dataset"]["dimension"][dimension_id]

    current_length = len(dimension["category"]["index"])
    nb = int(total_length / (length * current_length))

    v = list(json["dataset"]["value"])
    for x in range(length):
        for y, value in enumerate(dimension["category"]["label"].values()):
            for z in range(nb):
                v[x * current_length * nb + y * nb + z] = value

    values[dimension["label"]] = v

    length *= current_length

df = pd.DataFrame(values)
# print(json['dataset']['value'])
df

In [ ]:
import plotly.express as px

fig = px.line(
    df.query(f'Age == "99 years or older"'), x="Year", y="values", color="Age"
)
fig.update_layout(
    yaxis_type="log",
    xaxis={
        "rangeslider": {"visible": True},
    },
)
fig.show()